In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import torch

In [ ]:
!pip install transformers

     |████████████████████████████████| 675kB 7.6MB/s 
     |████████████████████████████████| 1.1MB 39.4MB/s 
     |████████████████████████████████| 890kB 39.8MB/s 
     |████████████████████████████████| 3.8MB 41.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=913ef26a9c0db02f453470ec537f4df5630898a724734ee1c86bc71c24899144
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from transformers import BertForSequenceClassification

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=4,
                                                      output_attentions=False,
                                                      output_hidden_states=False)



In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [ ]:
df=pd.read_csv('/content/drive/My Drive/processedData/rawData2020-03-23.csv.csv')

In [ ]:
df=df[['clean_text']]

In [ ]:
df

,clean_text
0,if such efforts r taken & increased samples r ...
1,time to boost up your defences flexed_biceps s...
2,we are not a corona we are indian India thumbs...
3,coronavirus has proved that everything around ...
4,"do not ignore ""science & technology"" only reli..."
...,...
4858,my prayers for minister saeed ghani who has te...
4859,being human by staying at home live and let li...
4860,not sure what is stopping you to declare total...
4861,can you imagine police in mp is forcing grocer...


In [ ]:
df=df.rename({'clean_text':'text'},axis=1)

In [ ]:
df

,text
0,if such efforts r taken & increased samples r ...
1,time to boost up your defences flexed_biceps s...
2,we are not a corona we are indian India thumbs...
3,coronavirus has proved that everything around ...
4,"do not ignore ""science & technology"" only reli..."
...,...
4858,my prayers for minister saeed ghani who has te...
4859,being human by staying at home live and let li...
4860,not sure what is stopping you to declare total...
4861,can you imagine police in mp is forcing grocer...


In [ ]:
encoded_data_test = tokenizer.batch_encode_plus(
    df.text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

In [ ]:
input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']


In [ ]:
dataset_test = TensorDataset(input_ids_test, attention_masks_test)

In [ ]:
len(dataset_test)

4863

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
batch_size = 32

dataloader_test = DataLoader(dataset_test, 
                              sampler=RandomSampler(dataset_test), 
                              batch_size=batch_size)

In [ ]:
def evaluate(dataloader_test):

    model.eval()
    
    
    predictions=[]
    #true_vals = []
    
    for batch in dataloader_test:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  #'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        logits=outputs[0]    
        logits=logits.detach().cpu().numpy()
        
        predictions.append(logits)


    
 
    
    predictions = np.concatenate(predictions, axis=0)
    #true_vals = np.concatenate(true_vals, axis=0)
            
    return predictions

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=4,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
model.load_state_dict(torch.load('/content/drive/My Drive/BertModel/finetuned_BERT_epoch_10.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
predictions=evaluate(dataloader_test)

In [ ]:
preds_flat = np.argmax(predictions,axis=1).flatten()

In [ ]:
df['sentiment']=pd.DataFrame(preds_flat)

In [ ]:
df

,text,sentiment
0,if such efforts r taken & increased samples r ...,1
1,time to boost up your defences flexed_biceps s...,0
2,we are not a corona we are indian India thumbs...,2
3,coronavirus has proved that everything around ...,1
4,"do not ignore ""science & technology"" only reli...",0
...,...,...
4858,my prayers for minister saeed ghani who has te...,1
4859,being human by staying at home live and let li...,0
4860,not sure what is stopping you to declare total...,1
4861,can you imagine police in mp is forcing grocer...,2


In [ ]:
label_dict_inverse={0:'sad', 1:'joy', 2:'fear', 3:'anger'}

In [ ]:
df=df.replace({"sentiment":label_dict_inverse})

In [ ]:
count=df['sentiment'].value_counts()

In [ ]:
df.loc[1,:]

text         time to boost up your defences flexed_biceps s...
sentiment                                                  sad
Name: 1, dtype: object

In [ ]:
# df.to_csv('/content/drive/My Drive/processedData/sentiment1.csv')